In [1]:
from multiprocessing import Pool
from IPython.display import clear_output
import os
import argparse


import jieba
import pandas as pd
import numpy as np


from procDataSet import TrainingQuery


top_num = 300

jieba.load_userdict(os.path.join('data', 'dict.txt.big'))
[jieba.add_word(i, freq=None, tag=None) for i in ['不支持','文林苑', '都更案','十八趴','證所稅','前瞻建設', '月退']]

# add NTUSD
positive_words = np.squeeze(pd.read_csv('data/NTUSD/ntusd-positive_zhtw.txt', header=None).values).tolist()
negative_words = np.squeeze(pd.read_csv('data/NTUSD/ntusd-negative_zhtw.txt', header=None).values).tolist()
for w in positive_words + negative_words:
    jieba.add_word(w)
    


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.497 seconds.
Prefix dict has been built succesfully.


In [2]:
with open('data/stop_word_zhtw.txt') as file:
    data = file.read()
    
stop_words = data.split('\n')
stop_words += ['「', '」', '，', '\n', '）', '（', ')', '(']

In [3]:
folder = 'data/'
raw_training_data = pd.read_csv(os.path.join(folder,'TD.csv'))
news_urls = pd.read_csv(os.path.join(folder,'NC_1.csv'))
contents = pd.read_json(os.path.join(folder,'url2content.json'), typ=pd.Series)
test_query = np.array(pd.read_csv('./data/QS_1.csv').Query)

## sort the contents by index
keys, content_list = contents.keys(), contents.values

In [4]:
# add back some words
stop_words = list(set(stop_words) - set(negative_words) - set(positive_words))

In [5]:
def find_sentiment(cuts):
    p = n = 0
    for c in cuts:
        p += c in positive_words
        n += c in negative_words
    return p - n

for q in test_query:
    cuts = jieba.lcut(q)
    cuts = [c for c in cuts if c not in stop_words]
    print(cuts)
    print(find_sentiment(cuts))

['通姦', '刑法', '應該', '除罪', '化']
-1
['應該', '取消', '機車', '強制', '二段式', '左轉', '待轉']
-3
['支持', '博弈', '特區', '台灣', '合法化']
0
['中華', '航空', '空服員', '罷工', '合理的']
0
['性交易', '應該', '合法化']
0
['ECFA', '早收', '清單', '達到', '預期', '成效']
1
['應該', '減免', '證所稅']
0
['贊成', '中油', '觀塘', '興建', '第三', '天然氣', '接收站']
1
['支持', '中國', '學生', '納入', '健保']
0
['支持', '臺灣', '中小學', '含', '高職', '專科', '服儀', '規定', '含', '髮', '襪', '鞋', '給予', '學生', '自主']
2
['不支持', '使用', '加密', '貨幣']
0
['不支持', '學雜費', '調漲']
0
['同意', '政府', '舉債', '發展', '前瞻建設', '計畫']
2
['支持', '電競', '列入', '體育競技']
0
['反對', '台鐵', '東移', '徵收', '案']
-2
['支持', '陳', '前', '總統', '保外就醫']
0
['年金', '改革', '應', '取消', '或應', '調降', '軍公教', '月退', '優存', '利率', '十八趴']
0
['同意', '動物', '實驗']
1
['油價', '應該', '凍漲', '緩漲']
0
['反對', '旺旺', '中時', '併購', '中嘉']
-1


In [ ]:
import multiprocessing

NumberCPU = multiprocessing.cpu_count()
jieba.initialize()

def jbcut(x):
    if x is not None:
        sen = jieba.lcut(x, cut_all=False)
        sen = [i for i in sen if i not in stop_words]
        return sen
    else:
        return None

def psegcut(x):
    if x is not None:
        sen = pseg.lcut(x)
        sen = [i for i in sen if i not in stop_words]
        return sen
    else:
        return None
    
pool = multiprocessing.Pool(processes=NumberCPU)
sentence_arr = pool.map(jbcut,content_list)
pool.close()
pool.join()

In [15]:
sentence_arr = np.load('data/sentence_arr.npy')
# np.save('data/sentence_arr', sentence_arr)

## add NTUSD scoring

In [163]:
# 要跑超久
pool = multiprocessing.Pool(processes=NumberCPU)
sentiment_arr = pool.map(find_sentiment, sentence_arr)
pool.close()
pool.join()


In [10]:
# np.save('data/sentiment_arr', sentiment_arr)
sentiment_arr = np.load('data/sentiment_arr.npy')

## okapi model

In [11]:
from gensim.summarization import bm25
bm25Model = bm25.BM25(sentence_arr)

In [20]:
train_query = sorted(set(raw_training_data.Query))

y_train = []
y_index = {}
for i  in train_query:
    index = np.where(raw_training_data.Query==i)
    data = raw_training_data.iloc[index]

    # sort by relvanance
    y = np.squeeze(data[data.Relevance!=0].sort_values('Relevance', ascending=False).News_Index.values).tolist()
    y_idx = [ (int(idx.split('_')[1])-1, rel )  for idx, rel in zip(data.News_Index, data.Relevance)]
    y_train.append(y)
    y_index[i] = [y_idx]
    
    print(i, len(y))

十二年國教高中職「免學費補助」適用對象增加是不對的 93
反對二代健保規定 232
反對旺旺中時併購中嘉 109
反對無圍牆校園 35
另立專法保障同婚是正確的 135
同意動物實驗 20
國際賽事會場內應該可以持中華民國國旗 766
堅決反對政府舉債發展前瞻建設計畫 388
年金改革應取消或應調降軍公教月退之優存利率十八趴 200
應該提高酒駕罰責以有效遏制酒駕 441
拒絕公投通過門檻下修 52
支持正名「臺灣」參與國際運動賽事 110
支持陳前總統保外就醫 200
核四應該啟用 205
油價應該凍漲或緩漲 68
臺灣應開放含瘦肉精(萊克多巴胺)之美國牛肉進口 248
贊同課綱微調 310
贊成文林苑都更案可依法拆除王家 65
贊成流浪動物零撲殺 270
遠雄大巨蛋工程應停工或拆除 240


In [22]:
total_scores = list()

for test_id , text_q in enumerate(train_query):
    text = jieba.lcut(text_q)
    text = [ t for t in text if t not in stop_words]
    scores = np.array(bm25Model.get_scores(text))
    scores = scores / np.amax(scores)
    top_query = np.argsort(scores)[::-1][:50]
      
    print(test_id ,text_q, text)
    
    # query expension
    for query in top_query:
        all_words = [ (sentence_arr[query][cnt], bm25Model.get_score(sentence_arr[query], cnt))
                     for cnt, i in enumerate(sentence_arr[query])]

        all_words = sorted(all_words,key=lambda x:(x[1]))[::-1]
        top_words = all_words[:20]
        top_words = [x[0] for x in top_words]
        text += top_words

    expended_score = np.array(bm25Model.get_scores(text))
    scores += expended_score / np.amax(expended_score)
    
    # scoring
    top_num = 300
    keys = pd.DataFrame(np.argsort(scores)[::-1][:top_num])
    ans = keys[0].apply(lambda x: 'news_{:06d}'.format(x+1))

    ap = sum([1 if a in y_train[test_id] else 0 for a in ans]) / min([300, len(y_train[test_id])])
    print("{:.2f}".format(ap))
    total_scores += [ap]

print('MAP: {:.2f}'.format(np.average(total_scores)))

0 十二年國教高中職「免學費補助」適用對象增加是不對的 ['十二年', '國教', '高中', '職', '免', '學費', '補助', '適用', '對象', '增加', '不對']
0.61
1 反對二代健保規定 ['反對', '二代', '健保', '規定']
0.56
2 反對旺旺中時併購中嘉 ['反對', '旺旺', '中時', '併購', '中嘉']
0.64
3 反對無圍牆校園 ['反對', '圍牆', '校園']
0.83
4 另立專法保障同婚是正確的 ['另立', '專法', '保障', '同婚', '正確的']
0.49
5 同意動物實驗 ['同意', '動物', '實驗']
1.00
6 國際賽事會場內應該可以持中華民國國旗 ['國際', '賽事', '會場', '應該', '持', '中華民國', '國旗']
0.33
7 堅決反對政府舉債發展前瞻建設計畫 ['堅決', '反對', '政府', '舉債', '發展', '前瞻建設', '計畫']
0.42
8 年金改革應取消或應調降軍公教月退之優存利率十八趴 ['年金', '改革', '應', '取消', '或應', '調降', '軍公教', '月退', '優存', '利率', '十八趴']
0.33
9 應該提高酒駕罰責以有效遏制酒駕 ['應該', '提高', '酒駕', '罰責以', '有效', '遏制', '酒駕']
0.65
10 拒絕公投通過門檻下修 ['拒絕', '公投', '通過', '門檻', '下修']
0.50
11 支持正名「臺灣」參與國際運動賽事 ['支持', '正名', '臺灣', '參與', '國際', '運動', '賽事']
0.61
12 支持陳前總統保外就醫 ['支持', '陳', '前', '總統', '保外就醫']
0.30
13 核四應該啟用 ['核四', '應該', '啟用']
0.18
14 油價應該凍漲或緩漲 ['油價', '應該', '凍漲', '緩漲']
0.54
15 臺灣應開放含瘦肉精(萊克多巴胺)之美國牛肉進口 ['臺灣', '應', '開放', '含', '瘦肉精', '萊克', '多巴胺', '美國', '牛肉', '進口']
0.24
16 贊同課綱微調 ['贊同', '課綱', '微調']
0.22
17 贊成文林苑都更案可依法

In [17]:
total_scores = list()

for test_id , text_q in enumerate(train_query):

    text = jieba.lcut(text_q)
    text = [ t for t in text if t not in stop_words]
    scores = np.array(bm25Model.get_scores(text))
    scores = scores / np.amax(scores)
    top_query = np.argsort(scores)[::-1][:50]
    
    sentiment_alpha = find_sentiment(text)
    
    print(test_id ,text_q, text, sentiment_alpha)
    
    # query expension
    for query in top_query:
        all_words = [ (sentence_arr[query][cnt], bm25Model.get_score(sentence_arr[query], cnt))
                     for cnt, i in enumerate(sentence_arr[query])]

        all_words = sorted(all_words,key=lambda x:(x[1]))[::-1]
        top_words = all_words[:20]
        top_words = [x[0] for x in top_words]
        text += top_words

    expended_score = np.array(bm25Model.get_scores(text))
    scores += expended_score / np.amax(expended_score)
        
    if sentiment_alpha != 0:
        sentiment = sentiment_alpha * np.sign(sentiment_arr)
        scores += sentiment / np.amax(np.abs(sentiment)) * .7
    
    # scoring
    top_num = 300
    keys = pd.DataFrame(np.argsort(scores)[::-1][:top_num])
    ans = keys[0].apply(lambda x: 'news_{:06d}'.format(x+1))

    ap = sum([1 if a in y_train[test_id] else 0 for a in ans]) / min([300, len(y_train[test_id])])
    print("{:.2f}".format(ap))
    total_scores += [ap]

print('MAP: {:.2f}'.format(np.average(total_scores)))

0 十二年國教高中職「免學費補助」適用對象增加是不對的 ['十二年', '國教', '高中', '職', '免', '學費', '補助', '適用', '對象', '增加', '不對'] 0
0.61
1 反對二代健保規定 ['反對', '二代', '健保', '規定'] -1
0.60
2 反對旺旺中時併購中嘉 ['反對', '旺旺', '中時', '併購', '中嘉'] -1
0.60
3 反對無圍牆校園 ['反對', '圍牆', '校園'] -1
0.43
4 另立專法保障同婚是正確的 ['另立', '專法', '保障', '同婚', '正確的'] 1
0.44
5 同意動物實驗 ['同意', '動物', '實驗'] 1
0.85
6 國際賽事會場內應該可以持中華民國國旗 ['國際', '賽事', '會場', '應該', '持', '中華民國', '國旗'] 0
0.33
7 堅決反對政府舉債發展前瞻建設計畫 ['堅決', '反對', '政府', '舉債', '發展', '前瞻建設', '計畫'] 1
0.32
8 年金改革應取消或應調降軍公教月退之優存利率十八趴 ['年金', '改革', '應', '取消', '或應', '調降', '軍公教', '月退', '優存', '利率', '十八趴'] 0
0.33
9 應該提高酒駕罰責以有效遏制酒駕 ['應該', '提高', '酒駕', '罰責以', '有效', '遏制', '酒駕'] 1
0.64
10 拒絕公投通過門檻下修 ['拒絕', '公投', '通過', '門檻', '下修'] 0
0.50
11 支持正名「臺灣」參與國際運動賽事 ['支持', '正名', '臺灣', '參與', '國際', '運動', '賽事'] 0
0.61
12 支持陳前總統保外就醫 ['支持', '陳', '前', '總統', '保外就醫'] 0
0.30
13 核四應該啟用 ['核四', '應該', '啟用'] 0
0.18
14 油價應該凍漲或緩漲 ['油價', '應該', '凍漲', '緩漲'] 0
0.54
15 臺灣應開放含瘦肉精(萊克多巴胺)之美國牛肉進口 ['臺灣', '應', '開放', '含', '瘦肉精', '萊克', '多巴胺', '美國', '牛肉', '進口'] 1
0.26
16 贊同課綱微調 ['贊

## Test the model

In [ ]:
total_scores = list()
for test_id , text_q in enumerate(test_query):
    text = jieba.lcut(text_q)
    text = [ t for t in text if t not in stop_words]
    scores = bm25Model.get_scores(text)
    top_query = np.argsort(scores)[::-1][:80]
    
    sentiment_alpha = find_sentiment(text)
    
    print(test_id ,text_q, text, sentiment_alpha)
    print("top query num {}".format(len(top_query)))
    
    # query expension
    for query in top_query:
        all_words = [ (sentence_arr[query][cnt], bm25Model.get_score(sentence_arr[query], cnt))
                     for cnt, i in enumerate(sentence_arr[query])]

        all_words = sorted(all_words,key=lambda x:(x[1]))[::-1]
        top_words = all_words[:20]
        top_words = [x[0] for x in top_words]
        text += top_words
    scores += 0.3 * np.array(bm25Model.get_scores(text))
    
#     if sentiment_alpha != 0:
#         sentiment = sentiment_alpha * np.sign(sentiment_arr)
#         scores += sentiment / np.amax(np.abs(sentiment)) * np.amax(scores) * .5

    # add train data
    delta = np.max(scores) 
    if text_q in y_index.keys():
        for idx, rel in (y_index[text_q][0]):
            scores[idx] += delta * idx
            
    total_scores.append(scores)

0 通姦在刑法上應該除罪化 ['通姦', '刑法', '應該', '除罪', '化'] -1
top query num 80
1 應該取消機車強制二段式左轉(待轉) ['應該', '取消', '機車', '強制', '二段式', '左轉', '待轉'] -3
top query num 80
2 支持博弈特區在台灣合法化 ['支持', '博弈', '特區', '台灣', '合法化'] 0
top query num 80
3 中華航空空服員罷工是合理的 ['中華', '航空', '空服員', '罷工', '合理的'] 0
top query num 80


In [24]:
top_num = 300
search_result = np.zeros((20,top_num))
for cnt,i in enumerate(total_scores):
    keys = np.argsort(i)[::-1][:top_num]
    search_result[cnt] += keys
    
search_result = search_result.astype(np.int)

df = pd.DataFrame()
df['Query_Index'] = ['q_{:02d}'.format(i+1) for i in range(20)]

for i in range(top_num):
    df['Rank_{:03d}'.format(i+1)] = search_result[:, i] + 1

for i, row in df.iterrows():
    df.iloc[i, 1:] = df.iloc[i, 1:].apply(lambda x: 'news_{:06d}'.format(x))

In [25]:
fname = '0625-2.csv'
df.to_csv('output/' + fname, index=False)